# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


In [5]:
# reading the file using panda so that we can aggregate and check queries later
df = pd.read_csv('event_datafile_new.csv', sep=',')
df.head()

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,The Grass Roots,Sara,F,72,Johnson,166.71302,paid,"Winston-Salem, NC",411,Let's Live For Today,95
1,Stars,Sara,F,73,Johnson,298.94485,paid,"Winston-Salem, NC",411,Time Can Never Kill The True Heart,95
2,Eddie Palmieri,Sara,F,74,Johnson,391.83628,paid,"Winston-Salem, NC",411,Nada De Ti,95
3,The Bravery,Sara,F,75,Johnson,168.14975,paid,"Winston-Salem, NC",411,Give In,95
4,K.U.K.L,Sara,F,76,Johnson,181.28934,paid,"Winston-Salem, NC",411,Anna,95


In [6]:
df[(df['song']=='All Hands Against His Own')].head()

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
270,The Black Keys,Jacqueline,F,50,Lynch,196.91057,paid,"Atlanta-Sandy Springs-Roswell, GA",559,All Hands Against His Own,29
916,The Black Keys,Tegan,F,25,Levine,196.91057,paid,"Portland-South Portland, ME",611,All Hands Against His Own,80
5267,The Black Keys,Sara,F,31,Johnson,196.91057,paid,"Winston-Salem, NC",152,All Hands Against His Own,95


### After we have retrived data from all the files, let us start modeling with Cassandra

#### Creating a Cluster

In [7]:
# This should make a connection to a Cassandra
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [8]:
#here is how we creat the database
session.execute("""
    CREATE KEYSPACE IF NOT EXISTS turki 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

#### Set Keyspace

In [9]:
# TO-DO: Set KEYSPACE to the keyspace specified above
session.set_keyspace('turki')

# Query 1

- Give me the artist, song title and song's length in the music app history that was heard during \
  sessionId = 338, and itemInSession = 4

### Creating

In [10]:
# let us use the session_id and item_in_session as the PRIMARY KEY

# here we need session_id and item_in_session as Primary Key \
# they consistitute a unique item and will be used in the where clause

q = 'CREATE TABLE IF NOT EXISTS songs'
q = q + '(session_id int, item_in_session int, artist text,   length float,  song text, PRIMARY KEY (session_id, item_in_session) )'


try:
    session.execute(q)
except Exception as e:
    print(e)                   

### Inserting

In [11]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#


# inserting values for the first table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        #write the insert query
        query = "INSERT INTO songs (session_id, item_in_session, artist,  length,  song)"
        query = query + "VALUES (%s,%s,%s,%s,%s)"
        #assign values to the query
        session.execute(query, (int(line[8]), int(line[3]), line[0],  float(line[5]),  line[9]))

### Testing

In [12]:
# let us evaluate our work thrugh the following query
query = 'SELECT artist, song, length FROM songs WHERE session_id=338 AND item_in_session = 4'
result = session.execute(query)

for row in result:
    print(row)

Row(artist='Faithless', song='Music Matters (Mark Knight Dub)', length=495.30731201171875)


Nice! it worked.

# Query 2

- Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
  for userid = 10, sessionid = 182

### Creating

In [13]:
# in this table we should include item_in_session so that we can sort on it

# This table Primary Key is chosen as user_id, session_id since the WHERE clause will be based on them, \
# and the item_in_session column is used for ordering
q = 'CREATE TABLE IF NOT EXISTS info'
q = q + '(user_id int, session_id int, item_in_session int, artist text,  first_name text,  last_name text,  song text,  PRIMARY KEY ((user_id, session_id),  item_in_session) )'


try:
    session.execute(q)
except Exception as e:
    print(e)                 

### Inserting

In [14]:
#this part is for iterating row by row in the data file

# now we will insert values to the second table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # write the insert query
        query = "INSERT INTO info (user_id, session_id, item_in_session, artist, first_name, last_name,  song )"
        query = query + "VALUES (%s,%s,%s,%s,%s,%s,%s)"
        #inser the values to the table
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[1], line[4], line[9] ))

### Testing

In [15]:
# now we select and evaluate our work

q = 'SELECT artist, song, first_name, last_name FROM info WHERE user_id=10 AND session_id=182'
result = session.execute(q)

for row in result:
    print(row)

Row(artist='Down To The Bone', song="Keep On Keepin' On", first_name='Sylvie', last_name='Cruz')
Row(artist='Three Drives', song='Greece 2000', first_name='Sylvie', last_name='Cruz')
Row(artist='Sebastien Tellier', song='Kilometer', first_name='Sylvie', last_name='Cruz')
Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', first_name='Sylvie', last_name='Cruz')


Cassandra is handling duplicates now, thanks for our Primary Key!

# Query 3
- Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

### Creating

In [32]:
# we will use song and user_id as the PRIMARY KEY

# here the Primary Key must be a composite of song and another column for easy search
# we will use song and user_id.
q = 'CREATE TABLE IF NOT EXISTS users'
q = q + '(song text, user_id int, first_name text,  last_name text, PRIMARY KEY (song, user_id))'


try:
    session.execute(q)
except Exception as e:
    print(e)                 

### Inserting

In [33]:
#this part is for iterating row by row in the data file

#now we insert for the third table songs3
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        #Assign the INSERT statements into the `query` variable
        query = "INSERT INTO users (song, user_id, first_name, last_name)"
        query = query + "VALUES (%s,%s,%s,%s)"
        # insert the corresponding values
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

### Testing

In [34]:
# evaluate query 3

q = "SELECT first_name, last_name FROM users WHERE song= 'All Hands Against His Own' "

result = session.execute(q)

for row in result:
    print(row)

Row(first_name='Jacqueline', last_name='Lynch')
Row(first_name='Tegan', last_name='Levine')
Row(first_name='Sara', last_name='Johnson')


Fine, clear result.

### Droping the tables 

In [31]:
#drop table 1
query = 'DROP TABLE IF EXISTS songs'
session.execute(query)

#drop table 2
query = 'DROP TABLE IF EXISTS info'
session.execute(query)

#drop table 3
query = 'DROP TABLE IF EXISTS users'
session.execute(query)

### Closing the session and cluster 

In [38]:
# close the connect
session.shutdown()
cluster.shutdown()